# XGB Model for predicting Churn 

## Importing Libraries

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

## Getting the data

In [3]:
import os 
os.chdir('../fake_CRM_data/')
data = pd.read_csv('placements.csv')
os.chdir('../models/')

In [5]:
data.head()

,Client,Placement Client Local ID,Placement Name,Coverage,Product Line,Carrier Group,Placement Created Date/Time,Placement Created By,Placement Created By (ID),Response Received Date,Placement Specialist,Placement Renewing Status,Placement Status,Declination Reason,Placement Id,Placement Effective Date,Placement Expiry Date,Incumbent Indicator,Participation Status Code,Placement Client Segment Code,Placement Renewing Status Code,Limit,Coverage Premium Amount,Tria Premium,Total Premium,Comission %,Comission Amount,Participation Percentage,Carrier Group Local ID,Production Code,Submission Sent Date,Program Product Local Code Text,Approach Non Admitted Market Indicator,Carrier Integration
0,Global Technologies,SCR-0b810b6f4c20,SCR-8d9f15ee3a3c,General Liability,Energy and Power,Eastern Risk Management,2025-04-24T06:37:09.314837765,Kimberly Jackson,SCR-c54656cdfecb,29/07/25,Mary Jackson,In progress,Quote,-,SCR-76fd0b40a1cb,30/09/25,30/09/26,N,QUOTATION_STATUS_QUOTED,CLIENT_SEGMENT_RISK_MGMT,RENEWAL_STATUS_IN_PROGRESS,3558700.0,65304.28,1881.62,67311.79,8,5760.41,100,0498,PRODUCTION_TYPE_NEW,-,SCR-262eac00ad8f,N,Not Applicable
1,Apex Enterprises,SCR-cfcd65ae0f7a,SCR-6b2e08c3f11d,General Liability,Casualty,Liberty Insurance Group,2025-04-25T15:21:40.575782415,Matthew Johnson,SCR-d47a7dd7c8ff,29/07/25,Donald Martin,In progress,Quote,-,SCR-07c81694713d,15/09/25,15/09/26,Y,QUOTATION_STATUS_QUOTED,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_IN_PROGRESS,82254400.0,1441557.71,55830.17,1581997.56,15,282637.79,100,0815,PRODUCTION_TYPE_RENEWAL,-,SCR-af66d3bd1f7b,Y,Not Applicable
2,Omega Global,SCR-51dd14cf0f45,SCR-6d2b42de057e,General Liability,Casualty,United Coverage Corp,2025-03-13T18:03:34.574937777,Robert Young,SCR-0b51d24e469e,-,Michelle Anderson,In progress,Submitted,-,SCR-b422393095aa,15/09/25,15/09/26,Y,QUOTATION_STATUS_SUBMITTED,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_IN_PROGRESS,225000.0,3279.01,0.00,3256.21,17,645.63,100,3093,PRODUCTION_TYPE_RENEWAL,29/07/25,SCR-af66d3bd1f7b,N,Not Applicable
3,Omega Global,SCR-51dd14cf0f45,SCR-6d2b42de057e,General Liability,Casualty,United Coverage Corp,2025-03-13T18:03:34.574937777,Robert Young,SCR-0b51d24e469e,-,Michelle Anderson,In progress,Submitted,-,SCR-b422393095aa,15/09/25,15/09/26,Y,QUOTATION_STATUS_SUBMITTED,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_IN_PROGRESS,225000.0,0.00,0.00,0.00,17,0.00,100,3093,PRODUCTION_TYPE_RENEWAL,29/07/25,SCR-af66d3bd1f7b,N,Not Applicable
4,Omega Global,SCR-51dd14cf0f45,SCR-6d2b42de057e,General Liability,Casualty,Northern Trust Insurance,2025-03-13T18:03:34.574937777,Robert Young,SCR-0b51d24e469e,-,Michelle Anderson,In progress,Submitted,-,SCR-b422393095aa,15/09/25,15/09/26,N,QUOTATION_STATUS_SUBMITTED,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_IN_PROGRESS,225000.0,0.00,0.00,0.00,17,0.00,100,4584,PRODUCTION_TYPE_RENEWAL,30/07/25,SCR-af66d3bd1f7b,N,Not Applicable


## Preprocessing

### to camel case

In [6]:
def camelCase(names: list) -> list:
  return [name\
          .strip() \
          .replace('_', ' ') \
        #   .title() \
          .replace(' ', '') \
          .replace('(', '_') \
          .replace(')', '') \
          for name in names]

data.columns = camelCase(data.columns)
data.rename({'PlacementCreatedDate/Time': 'PlacemenCreatedDatetime'}, inplace=True)

### casting '-' with NA

In [7]:
for col in data.columns:
    data[col] = data[col].replace('-', pd.NA)

### to datetime 

In [8]:
def setColsToDatetime(df: pd.DataFrame, cols: list) -> None:
    for col in cols:
        df[col] = pd.to_datetime(df[col], dayfirst=True, errors='ignore')

In [9]:
setColsToDatetime(
    df=data, 
    cols=\
        ['ResponseReceivedDate', 'PlacementEffectiveDate',
       'PlacementExpiryDate', 'SubmissionSentDate']
)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3668\3892670953.py:3: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_datetime(df[col], dayfirst=True, errors='ignore')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3668\3892670953.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], dayfirst=True, errors='ignore')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3668\3892670953.py:3: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_datetime(df[col], dayfirst=True, errors='ignore')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3668\3892670953.py:3: UserWarning: Could n

## Adding Columns

In [10]:
data['_DaysToExpiry'] = (data.PlacementExpiryDate - data.PlacementEffectiveDate).apply(lambda x: x.days)
data['_CarrierResponseTime'] = (data.ResponseReceivedDate - data.SubmissionSentDate).apply(lambda x: x.days)

## Preparing Data for Training 

In [11]:
data.head()

,Client,PlacementClientLocalID,PlacementName,Coverage,ProductLine,CarrierGroup,PlacementCreatedDate/Time,PlacementCreatedBy,PlacementCreatedBy_ID,ResponseReceivedDate,PlacementSpecialist,PlacementRenewingStatus,PlacementStatus,DeclinationReason,PlacementId,PlacementEffectiveDate,PlacementExpiryDate,IncumbentIndicator,ParticipationStatusCode,PlacementClientSegmentCode,PlacementRenewingStatusCode,Limit,CoveragePremiumAmount,TriaPremium,TotalPremium,Comission%,ComissionAmount,ParticipationPercentage,CarrierGroupLocalID,ProductionCode,SubmissionSentDate,ProgramProductLocalCodeText,ApproachNonAdmittedMarketIndicator,CarrierIntegration,_DaysToExpiry,_CarrierResponseTime
0,Global Technologies,SCR-0b810b6f4c20,SCR-8d9f15ee3a3c,General Liability,Energy and Power,Eastern Risk Management,2025-04-24T06:37:09.314837765,Kimberly Jackson,SCR-c54656cdfecb,2025-07-29,Mary Jackson,In progress,Quote,<NA>,SCR-76fd0b40a1cb,2025-09-30,2026-09-30,N,QUOTATION_STATUS_QUOTED,CLIENT_SEGMENT_RISK_MGMT,RENEWAL_STATUS_IN_PROGRESS,3558700.0,65304.28,1881.62,67311.79,8,5760.41,100,0498,PRODUCTION_TYPE_NEW,NaT,SCR-262eac00ad8f,N,Not Applicable,365,NaN
1,Apex Enterprises,SCR-cfcd65ae0f7a,SCR-6b2e08c3f11d,General Liability,Casualty,Liberty Insurance Group,2025-04-25T15:21:40.575782415,Matthew Johnson,SCR-d47a7dd7c8ff,2025-07-29,Donald Martin,In progress,Quote,<NA>,SCR-07c81694713d,2025-09-15,2026-09-15,Y,QUOTATION_STATUS_QUOTED,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_IN_PROGRESS,82254400.0,1441557.71,55830.17,1581997.56,15,282637.79,100,0815,PRODUCTION_TYPE_RENEWAL,NaT,SCR-af66d3bd1f7b,Y,Not Applicable,365,NaN
2,Omega Global,SCR-51dd14cf0f45,SCR-6d2b42de057e,General Liability,Casualty,United Coverage Corp,2025-03-13T18:03:34.574937777,Robert Young,SCR-0b51d24e469e,NaT,Michelle Anderson,In progress,Submitted,<NA>,SCR-b422393095aa,2025-09-15,2026-09-15,Y,QUOTATION_STATUS_SUBMITTED,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_IN_PROGRESS,225000.0,3279.01,0.00,3256.21,17,645.63,100,3093,PRODUCTION_TYPE_RENEWAL,2025-07-29,SCR-af66d3bd1f7b,N,Not Applicable,365,NaN
3,Omega Global,SCR-51dd14cf0f45,SCR-6d2b42de057e,General Liability,Casualty,United Coverage Corp,2025-03-13T18:03:34.574937777,Robert Young,SCR-0b51d24e469e,NaT,Michelle Anderson,In progress,Submitted,<NA>,SCR-b422393095aa,2025-09-15,2026-09-15,Y,QUOTATION_STATUS_SUBMITTED,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_IN_PROGRESS,225000.0,0.00,0.00,0.00,17,0.00,100,3093,PRODUCTION_TYPE_RENEWAL,2025-07-29,SCR-af66d3bd1f7b,N,Not Applicable,365,NaN
4,Omega Global,SCR-51dd14cf0f45,SCR-6d2b42de057e,General Liability,Casualty,Northern Trust Insurance,2025-03-13T18:03:34.574937777,Robert Young,SCR-0b51d24e469e,NaT,Michelle Anderson,In progress,Submitted,<NA>,SCR-b422393095aa,2025-09-15,2026-09-15,N,QUOTATION_STATUS_SUBMITTED,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_IN_PROGRESS,225000.0,0.00,0.00,0.00,17,0.00,100,4584,PRODUCTION_TYPE_RENEWAL,2025-07-30,SCR-af66d3bd1f7b,N,Not Applicable,365,NaN


In [13]:
data.PlacementRenewingStatusCode.value_counts()

PlacementRenewingStatusCode
RENEWAL_STATUS_FULLY_RENEWED    703
RENEWAL_STATUS_IN_PROGRESS      470
RENEWAL_STATUS_PART_RENEWED     173
RENEWAL_STATUS_OUT_OF_SCOPE       2
Name: count, dtype: int64